# 1: Setup Environment

In [ ]:
lat = 32.990254 #launch site latitude
long = -106.974998 #launch site longitude
elev = 1400 #launch site elevation (ASL)
days_from_today = 0 # >= 0
hour = 12 #in UTC
max_height = 10000



from rocketpy import Environment

env = Environment(latitude=lat, longitude=long, elevation=elev, max_expected_height=max_height)

import datetime
date = datetime.date.today() + datetime.timedelta(days=days_from_today)

env.set_date(
    (date.year, date.month, date.day, hour)
)

env.set_atmospheric_model(type="Ensemble", file="GEFS")

#### Display Environment Details:

In [ ]:
env.info()

# 2: Setup Engine (Solid Fuel)

In [ ]:
from rocketpy import SolidMotor
CosmoEngine = SolidMotor(
    thrust_source = lambda x : 2137/(x*x*x+1), #number, lambda, CSV file (time(s), thrust(N)), eng file from provider
    dry_mass=1.815, #mass of the engine in kg
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=33 / 1000,
    grain_number=5,
    grain_density=1815,
    grain_outer_radius=33 / 1000,
    grain_initial_inner_radius=15 / 1000,
    grain_initial_height=120 / 1000,
    grain_separation=5 / 1000,
    grains_center_of_mass_position=0.397,
    center_of_dry_mass_position=0.317,
    nozzle_position=0,
    burn_time=3.9,
    throat_radius=11 / 1000,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

#### Display Info and Draw:

In [ ]:
CosmoEngine.info()
CosmoEngine.draw()

#### Display Full Info Dump:

In [ ]:
CosmoEngine.all_info()

# 3: Setup Rocket

In [ ]:
from rocketpy import Rocket
CosmoRocket = Rocket(
    radius=127 / 2000,
    mass=14.426, #in kg, without engine, with everything else ex.(payload, electronics)
    inertia=(6.321, 6.321, 0.034),
    power_off_drag= lambda x : x, # CSV file(Mach, Drag), function(Mach) = Drag #RASAero II
    power_on_drag= lambda x : x, # CSV file(Mach, Drag), function(Mach) = Drag  #RASAero II
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

#### Add active engine to rocket:

In [ ]:
CosmoRocket.add_motor(CosmoEngine, position=-1.255)

#### Define and Add rail buttons:

In [ ]:
rail_buttons = CosmoRocket.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.6182,
    angular_position=45,
)

#### Define and Add nose cone

In [ ]:
nose_cone = CosmoRocket.add_nose(
    length=0.55829,
    kind="von karman", # von karman, conical, ogive, Ivhaack, powerseries
    position=1.278,
    #bluffness = 1 # (for ogive type nose) ratio between radius on the tip and the base
    #power = 1 # (for powerseries type nose)
)

#### Define and Add tapezoidal fins:

In [ ]:
fin_set = CosmoRocket.add_trapezoidal_fins(
    n=4, # > 2
    span=0.110,
    root_chord=0.120,
    tip_chord=0.060,
    position=-1.04956,
    cant_angle=0.5, #in degrees
    #airfoil = ("file","radians")#CSV or txt file (angle OA, lift) #airfoiltools.com
)

#### Define and Add elliptical fins:

In [ ]:
fin_set = CosmoRocket.add_elliptical_fins(
    n=4, # > 2
    span=0.110,
    root_chord=0.120,
    position=-1.04956,
    cant_angle=0.5, #in degrees
    #airfoil = ("file","radians")#CSV or txt file (angle OA, lift) #airfoiltools.com
)

#### Define and Add tail:

In [ ]:
tail = CosmoRocket.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)

#### Define and Add parachute:

In [ ]:
main = CosmoRocket.add_parachute(
    name="main",
    cd_s=10.0, #drag * parachute area
    trigger=800,      # altitude in meters, "apogee", lambda (p, h, y[x,y,z,vx,vy,vz,e0,e1,e2,e3,w1,w2,w3])
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

#### Display rocket info: (if a mistake was made please return to step 1, 2 or 3)

In [ ]:
CosmoRocket.info()

#### Check if rocket is stable: (negative or very high values might fail the simulation)

In [ ]:
CosmoRocket.plots.static_margin()

#### Display current rocket:

In [ ]:
CosmoRocket.draw()

# 4: Simulation
#### Start simulation: (name it distinctly if for comparison uses)

In [ ]:
from rocketpy import Flight
test_flight = Flight(
    rocket=CosmoRocket, environment=env, rail_length=5.2, inclination=85, heading=0, name="flight"
)

#### Initiate and clear comparison list: (optional)

In [ ]:
compare_list = []

#### Add this simulation to comparison list: (optional)

In [ ]:
compare_list.append(test_flight)

#### Plot simulated trajectory:

In [ ]:
test_flight.plots.trajectory_3d()

#### Display basic flight info:

In [ ]:
test_flight.info()

#### Display velocity plots:

In [ ]:
test_flight.plots.linear_kinematics_data()

#### Display flight angle plots:

In [ ]:
test_flight.plots.flight_path_angle_data() #the more apart angle lines are the less stable rocket is

#### Dump all flight info:

In [ ]:
test_flight.all_info() #additional info dump

## Compare added flights:

In [ ]:
from rocketpy.plots.compare import CompareFlights

comparison = CompareFlights(compare_list)

comparison.trajectories_3d(legend=True)

#### Draw basic info:

In [ ]:
comparison.positions()
comparison.velocities()
comparison.accelerations()

#### Dump all comparison info:

In [ ]:
comparison.all()

## Mission Type: Payload
#### Define Payload:

In [ ]:
from rocketpy import Rocket

payload = Rocket(
    radius=127 / 2000,
    mass=1,
    inertia=(0.1, 0.1, 0.001),
    power_off_drag=0.5,
    power_on_drag=0.5,
    center_of_mass_without_motor=0,
)


CosmoRocketNoPayload = CosmoRocket
CosmoRocketNoPayload.mass = CosmoRocketNoPayload.mass - payload.mass

#### Define and Add payload parachute:

In [ ]:
payload_main = payload.add_parachute(
    "Main",
    cd_s=4.0,
    trigger=400,
    sampling_rate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

#### Simulate Payload Mission: (remember to set rail length, inclination and direction in phase 1)

In [ ]:
from rocketpy.plots.compare import CompareFlights

phase1 = Flight(rocket=CosmoRocket, environment=env, rail_length=5.2, inclination=85, heading=0, terminate_on_apogee=True, name="Rocket with Payload")
phase2 = Flight(rocket=payload, environment=env, rail_length=5.2, inclination=85, heading=0, initial_solution=phase1, name="Payload")
phase3 = Flight(rocket=CosmoRocketNoPayload, environment=env, rail_length=5.2, inclination=85, heading=0, initial_solution=phase1, name="Rocket without Payload")

comparison = CompareFlights([phase1, phase2, phase3])

#### Plot Mission Trajectory:

In [ ]:
comparison.trajectories_3d(legend=True)

#### Draw basic info:

In [ ]:
comparison.positions()
comparison.velocities()
comparison.accelerations()

#### Dump all mission info

In [ ]:
comparison.all()

## Mission Type: Uncertain Values
#### Define Uncertain Environment:

In [ ]:
from rocketpy.stochastic import StochasticEnvironment

#set how much you want the value to change (using normal distribution)
#if some values aren't meant to be randomize, comment these lines
random_env = StochasticEnvironment(
    environment=env,
    ensemble_member=list(range(env.num_ensemble_members)), #randomly select from few forecasts, if unwanted comment this line
    longitude = 0.00001,
    latitude = 0.00001,
    elevation = 1
)

#### Show Environment Randomization:

In [ ]:
random_env.visualize_attributes()

#### Define Uncertain Engine:

In [ ]:
from rocketpy.stochastic import StochasticSolidMotor

#set how much you want the value to change (using normal distribution)
#if some values aren't meant to be randomize, comment these lines
random_Engine = StochasticSolidMotor(
    solid_motor = CosmoEngine,
    burn_out_time=0.1,
    burn_start_time=0.1,
    dry_mass=0.01,
    center_of_dry_mass_position=0.001,
    grains_center_of_mass_position=0.001,
    grain_density=50,
    grain_separation=1 / 1000,
    grain_initial_height=1 / 1000,
    grain_initial_inner_radius=0.375 / 1000,
    grain_outer_radius=0.375 / 1000,
    total_impulse=5,
    throat_radius=0.5 / 1000,
    nozzle_radius=0.5 / 1000,
    nozzle_position=0.001,
)

#### Show Engine Randomization:

In [ ]:
random_Engine.visualize_attributes()

#### Define Uncertain Rocket:

In [ ]:
from rocketpy.stochastic import StochasticRocket

#set how much you want the value to change (using normal distribution)
#if some values aren't meant to be randomize, comment these lines
random_rocket = StochasticRocket(
    rocket=CosmoRocket,
    radius=0.0127 / 2000,
    mass=0.05,
    inertia_11=0.01,
    inertia_22=0.01,
    inertia_33=0.01,
    center_of_mass_without_motor=0.1,
)
random_rocket.add_motor(random_Engine, position=0.01)

#### Show Rocket Randomization:

In [ ]:
random_rocket.visualize_attributes()

#### Define Uncertain Flight: 

In [ ]:
from rocketpy.stochastic import StochasticFlight

#set how much you want the value to change (using normal distribution)
#if some values aren't meant to be randomize, comment these lines
random_flight = StochasticFlight(
    flight=test_flight,
    rail_length=0.1,
    inclination=2,
    heading=1
)

#### Show Flight Randomization:

In [ ]:
random_flight.visualize_attributes()

#### Simulate (n) Randomized Flights: (might take a while!)

In [ ]:
n = 100

from rocketpy import MonteCarlo

simulations = MonteCarlo(
    filename="monte_carlo_simulations/test", #directory must exist!
    environment=random_env,
    rocket=random_rocket,
    flight=random_flight
)

simulations.simulate(
    number_of_simulations=n, append=False, include_function_data=False
)

#### Show Results:

In [ ]:
simulations.prints.all()
simulations.plots.all()
#apogee is in ASL not AGL!!!